In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [3]:
import malaya

model = malaya.translation.en_ms.transformer(model = 'noisy-base')

/home/ubuntu/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


Downloading:   0%|          | 0.00/233M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234k [00:00<?, ?B/s]

2022-10-04 11:41:59.749385: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-04 11:41:59.765725: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-04 11:41:59.765741: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-10-04 11:41:59.765746: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-10-04 11:41:59.765800: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-10-04 11:41:59.765825: I

In [6]:
from unidecode import unidecode

with open('eng_Latn.dev') as fopen:
    eng = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
right = [unidecode(s) for s in ms]
left = [unidecode(s) for s in eng]

In [12]:
from tqdm import tqdm

batch_size = 2

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 499/499 [02:40<00:00,  3.11it/s]


In [13]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [14]:
refs = [filtered_right]
sys = filtered_left

In [15]:
r = bleu.corpus_score(sys, refs)
r.__dict__

{'name': 'BLEU',
 'score': 41.949315647096626,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '73.4/50.3/35.7/25.7 (BP = 0.977 ratio = 0.977 hyp_len = 21526 ref_len = 22027)',
 'bp': 0.9769945746149022,
 'counts': [15810, 10330, 6977, 4772],
 'totals': [21526, 20529, 19532, 18535],
 'sys_len': 21526,
 'ref_len': 22027,
 'precisions': [73.44606522345072,
  50.31906084076185,
  35.720868318656564,
  25.745886161316427],
 'prec_str': '73.4/50.3/35.7/25.7',
 'ratio': 0.9772551868161802}

In [16]:
chrf.corpus_score(sys, refs)

chrF2++ = 66.20